In [1]:
import pandas
import geopandas 
import feedparser
import requests
from   zipfile import ZipFile
import datetime 
from   local_module import NhcDownloaderBot

%matplotlib inline

In [2]:
url = 'https://www.nhc.noaa.gov/index-at.xml'
f = feedparser.parse(url)
f

{'bozo': False,
 'entries': [{'title': 'Atlantic Tropical Weather Outlook',
   'title_detail': {'type': 'text/plain',
    'language': None,
    'base': 'https://www.nhc.noaa.gov/index-at.xml',
    'value': 'Atlantic Tropical Weather Outlook'},
   'summary': '<br />\n000<br />\nABNT20 KNHC 281723<br />\nTWOAT <br />\n<br />\nTropical Weather Outlook<br />\nNWS National Hurricane Center Miami FL<br />\n200 PM EDT Tue Sep 28 2021<br />\n<br />\nFor the North Atlantic...Caribbean Sea and the Gulf of Mexico:<br />\n<br />\nThe National Hurricane Center is issuing advisories on Hurricane <br />\nSam, located several hundred miles east of the northern Leeward <br />\nIslands.<br />\n<br />\nShowers and thunderstorms remain disorganized in association with a <br />\na broad area of low pressure located several hundred miles <br />\nsouthwest of the Cabo Verde Islands. Environmental conditions are <br />\nconducive for gradual development, and a tropical depression is <br />\nlikely to form dur

In [3]:
#f
#f.keys()
# len(f.entries)
# f['entries'][1]['nhc_atcf']
# f.status
# f.feed
# f.feed['subtitle']
# f.entries
# type(f.entries)
# for entry in f.entries:
#     print(entry.title)

In [3]:
df = pandas.DataFrame(f.entries)
df = df.drop(columns=['title_detail', 'summary', 'summary_detail', 'published_parsed', 'links', 'link', 'id', 
                      'guidislink', 'authors', 'author', 'author_detail'])
df.head(5)

,title,published,nhc_center,nhc_type,nhc_name,nhc_wallet,nhc_atcf,nhc_datetime,nhc_movement,nhc_pressure,nhc_wind,nhc_headline,nhc_cyclone
0,Atlantic Tropical Weather Outlook,"Tue, 28 Sep 2021 17:24:04 GMT",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Summary for Hurricane Sam (AT3/AL182021),"Tue, 28 Sep 2021 14:32:57 GMT","17.5, -54.3",Hurricane,Sam,AT3,AL182021,11:00 AM AST Tue Sep 28,NW at 8 mph,952 mb,130 mph,...SAM REMAINS A POWERFUL CATEGORY 4 HURRICANE...,
2,Hurricane Sam Public Advisory Number 24,"Tue, 28 Sep 2021 14:32:57 GMT",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Hurricane Sam Forecast Advisory Number 24,"Tue, 28 Sep 2021 14:32:26 GMT",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Hurricane Sam Forecast Discussion Number 24,"Tue, 28 Sep 2021 14:33:52 GMT",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
tc_list = [nhc_atcf[2:4] for nhc_atcf in df['nhc_atcf'] if pandas.isnull(nhc_atcf) == False] 
tc_list

['17', '18']

In [6]:
#{key: value for (key, value) in iterable}
d = {nhc_atcf: [nhc_name    , nhc_type, nhc_center, nhc_movement, 
                nhc_pressure, nhc_wind, published , nhc_datetime]         
     for (nhc_name    , nhc_atcf, nhc_type  , nhc_center  , nhc_movement,
          nhc_pressure, nhc_wind, published , nhc_datetime) 
     in  zip(df['nhc_name']    , df['nhc_atcf']    , df['nhc_type'], df['nhc_center'], 
             df['nhc_movement'], df['nhc_pressure'], df['nhc_wind'], df['published'],
             df['nhc_datetime']) 
     if pandas.isnull(nhc_name) == False}   
d

{'AL172021': ['Rose',
  'POST-TROPICAL CYCLONE',
  '25.2, -41.6',
  'NW at 10 mph',
  '1010 mb',
  '35 mph',
  'Thu, 23 Sep 2021 08:32:17 GMT',
  '9:00 AM GMT Thu Sep 23'],
 'AL182021': ['Eighteen',
  'Tropical Depression',
  '10.5, -36.4',
  'W at 15 mph',
  '1008 mb',
  '35 mph',
  'Thu, 23 Sep 2021 08:34:16 GMT',
  '5:00 AM AST Thu Sep 23']}

In [26]:
class NhcRssParser:
    def __init__(self, url):    
        self.url   = url 
        self.f     = f   =  feedparser.parse(self.url)
        self.df    = df  =  pandas.DataFrame(self.f.entries).drop(columns=['title_detail', 'summary', 'summary_detail', 'published_parsed', 'links', 'link', 'id', 
                               'guidislink', 'authors', 'author', 'author_detail'])
    
    def tc_list(self):
        return [nhc_atcf for nhc_atcf in self.df['nhc_atcf'] if pandas.isnull(nhc_atcf) == False]  
    #{key: value for (key, value) in iterable}
    def tc_dict(self):
        return {nhc_atcf: [nhc_name    , nhc_type, nhc_center, nhc_movement, 
                        nhc_pressure, nhc_wind, published , nhc_datetime]         
             for (nhc_name    , nhc_atcf, nhc_type  , nhc_center  , nhc_movement,
                  nhc_pressure, nhc_wind, published , nhc_datetime) 
             in  zip(self.df['nhc_name']    , self.df['nhc_atcf']    , self.df['nhc_type'], self.df['nhc_center'], 
                     self.df['nhc_movement'], self.df['nhc_pressure'], self.df['nhc_wind'], self.df['published'],
                     self.df['nhc_datetime']) 
             if pandas.isnull(nhc_name) == False}

url = 'https://www.nhc.noaa.gov/index-at.xml'
a = NhcRssParser(url)
a.df

list_ = a.tc_list()
dict_ = a.tc_dict()
for i in range(len(list_)):
    print(dict_[list_[i]][2])


10.9, -38.1


In [3]:
class NhcRssParser:
    def __init__(self, url):    
        self.url   = url 
        self.f     = f   =  feedparser.parse(self.url)
        self.df    = df  =  pandas.DataFrame(self.f.entries).drop(columns=['title_detail', 'summary', 'summary_detail', 'published_parsed', 'links', 'link', 'id', 
                               'guidislink', 'authors', 'author', 'author_detail'])
    
    #{key: value for (key, value) in iterable}
    def tc_dict_list(self):
        return {nhc_atcf: [nhc_name    , nhc_type, nhc_center, nhc_movement, 
                        nhc_pressure, nhc_wind, published , nhc_datetime]         
             for (nhc_name    , nhc_atcf, nhc_type  , nhc_center  , nhc_movement,
                  nhc_pressure, nhc_wind, published , nhc_datetime) 
             in  zip(self.df['nhc_name']    , self.df['nhc_atcf']    , self.df['nhc_type'], self.df['nhc_center'], 
                     self.df['nhc_movement'], self.df['nhc_pressure'], self.df['nhc_wind'], self.df['published'],
                     self.df['nhc_datetime']) 
             if pandas.isnull(nhc_name) == False}, [nhc_atcf for nhc_atcf in self.df['nhc_atcf'] if pandas.isnull(nhc_atcf) == False]
url    = 'https://www.nhc.noaa.gov/index-at.xml'
tcdict, tclist = NhcRssParser(url).tc_dict_list()

In [ ]:
tcdict[tclist[0]][1] = tcdictgral['nhc_type'][1][tcdictgral['nhc_type'][0].index(tcdict[tclist[0]][1].casefold())]   
tcdict

In [19]:
class NhcDownloaderBot:
    def __init__(self, storm_number, year):
        self.storm_number = storm_number
        self.year         = year
        
        
        self.file_names     = file_names = [
                            f'al{self.storm_number}{self.year}_5day_latest.zip',
                            f'al{self.storm_number}{self.year}_fcst_latest.zip',
                            f'al{self.storm_number}{self.year}_best_track.zip',
                            f'gtwo_shapefiles.zip',
                            f'wsp_120hrhalfDeg_latest.zip',
                            f'wsp_120hr5km_latest.zip'
                            ]
        self.urls           = urls = [
                            f'https://www.nhc.noaa.gov/gis/forecast/archive',
                            f'https://www.nhc.noaa.gov/gis/forecast/archive',
                            f'https://www.nhc.noaa.gov/gis/best_track',
                            f'https://www.nhc.noaa.gov/xgtwo',
                            f'https://www.nhc.noaa.gov/gis/forecast/archive',
                            f'https://www.nhc.noaa.gov/gis/forecast/archive'
                            ]
        self.gdf_names      = gdf_names = {
                            file_names[0] : ['track_line_gdf', 'cone_gdf', 'points_gdf'],
                            file_names[1] : ['init_radii_gdf', 'fcst_radii_gdf'],
                            file_names[2] : ['best_track_points_gdf', 'best_track_line_gdf', 'best_track_radii_gdf', 'best_track_swath_gdf'],
                            file_names[3] : ['gtwo_areas_gdf', 'gtwo_lines_gdf', 'gtwo_points_gdf'],
                            file_names[4] : ['wsp_34_gdf_points', 'wsp_50_gdf_points', 'wsp_64_gdf_points'],
                            file_names[5] : ['wsp_34_gdf_polygons', 'wsp_50_gdf_polygons', 'wsp_64_gdf_polygons']
                                          }
    def nhc_gis_downloader(self, file_names, urls, gdf_names):

        for idx, file_name in enumerate(file_names):
            url = f'{urls[idx]}/{file_name}'
            r = requests.get(url)
            with open(f'nhc_latest/{file_name}', 'wb') as code:
                code.write(r.content) 
        _  = [0, 3, 4, 5]  
        __ = [2, 7, 12]  
        for idx, file_name in enumerate([file_names[0],  file_names[3], 
                                        file_names[4],  file_names[5]]): 
            with ZipFile(f'nhc_latest/{file_name}', 'r') as zip_file_name:
                for idx2 in range(len(gdf_names[file_names[_[idx]]])):
                    gdf_names[file_names[_[idx]]][idx2]  = geopandas.read_file(
                    f'zip://./nhc_latest/{file_name}!{zip_file_name.namelist()[__[idx2]]}')
        _  = [0,5]                                 
        with ZipFile(f'nhc_latest/{file_names[1]}', 'r') as zip_file_name:
                for idx in range(2):
                    gdf_names[file_names[1]][idx]  = geopandas.read_file(
                    f'zip://./nhc_latest/{file_names[1]}!{zip_file_name.namelist()[_[idx]]}')
        _  = [2, 7, 13, 16]                                 
        with ZipFile(f'nhc_latest/{file_names[2]}', 'r') as zip_file_name:
            for idx in range(4):
                try:
                    gdf_names[file_names[2]][idx]  = geopandas.read_file(
                    f'zip://./nhc_latest/{file_names[2]}!{zip_file_name.namelist()[_[idx]]}')
                except IndexError:
                    gdf_names[file_names[2]][idx] = None
                    
        return gdf_names
    
    @classmethod
    def gdf_file(cls, storm_number, year):
        cls._                     = NhcDownloaderBot(storm_number, year)
        cls.track_line_gdf        = cls._.nhc_gis_downloader(cls._.file_names, cls._.urls, cls._.gdf_names)[cls._.file_names[0]][0]
        cls.cone_gdf              = cls._.nhc_gis_downloader(cls._.file_names, cls._.urls, cls._.gdf_names)[cls._.file_names[0]][1]
        cls.points_gdf            = cls._.nhc_gis_downloader(cls._.file_names, cls._.urls, cls._.gdf_names)[cls._.file_names[0]][2]
        cls.init_radii_gdf        = cls._.nhc_gis_downloader(cls._.file_names, cls._.urls, cls._.gdf_names)[cls._.file_names[1]][0]
        cls.fcst_radii_gdf        = cls._.nhc_gis_downloader(cls._.file_names, cls._.urls, cls._.gdf_names)[cls._.file_names[1]][1]
        cls.best_track_points_gdf = cls._.nhc_gis_downloader(cls._.file_names, cls._.urls, cls._.gdf_names)[cls._.file_names[2]][0]
        cls.best_track_line_gdf   = cls._.nhc_gis_downloader(cls._.file_names, cls._.urls, cls._.gdf_names)[cls._.file_names[2]][1]
        cls.best_track_radii_gdf  = cls._.nhc_gis_downloader(cls._.file_names, cls._.urls, cls._.gdf_names)[cls._.file_names[2]][2]
        cls.best_track_swath_gdf  = cls._.nhc_gis_downloader(cls._.file_names, cls._.urls, cls._.gdf_names)[cls._.file_names[2]][3]
        cls.gtwo_areas_gdf        = cls._.nhc_gis_downloader(cls._.file_names, cls._.urls, cls._.gdf_names)[cls._.file_names[3]][0]
        cls.gtwo_lines_gdf        = cls._.nhc_gis_downloader(cls._.file_names, cls._.urls, cls._.gdf_names)[cls._.file_names[3]][1]
        cls.gtwo_points_gdf       = cls._.nhc_gis_downloader(cls._.file_names, cls._.urls, cls._.gdf_names)[cls._.file_names[3]][2]
        cls.wsp_34_gdf_points     = cls._.nhc_gis_downloader(cls._.file_names, cls._.urls, cls._.gdf_names)[cls._.file_names[4]][0]
        cls.wsp_50_gdf_points     = cls._.nhc_gis_downloader(cls._.file_names, cls._.urls, cls._.gdf_names)[cls._.file_names[4]][1]
        cls.wsp_64_gdf_points     = cls._.nhc_gis_downloader(cls._.file_names, cls._.urls, cls._.gdf_names)[cls._.file_names[4]][2]
        cls.wsp_34_gdf_polygons   = cls._.nhc_gis_downloader(cls._.file_names, cls._.urls, cls._.gdf_names)[cls._.file_names[5]][0]
        cls.wsp_50_gdf_polygons   = cls._.nhc_gis_downloader(cls._.file_names, cls._.urls, cls._.gdf_names)[cls._.file_names[5]][1]
        cls.wsp_64_gdf_polygons   = cls._.nhc_gis_downloader(cls._.file_names, cls._.urls, cls._.gdf_names)[cls._.file_names[5]][2]

In [20]:
NhcDownloaderBot.gdf_file(18, 2021)
print('Done')

Done


In [25]:
NhcDownloaderBot.best_track_swath_gdf

,RADII,STORMID,BASIN,STORMNUM,ADVNUM,SYNOPTIME,TIMEZONE,NE,SE,SW,NW,geometry
0,34.0,AL182021,al,18.0,4.0,2021092306,UTC,30.0,0.0,0.0,30.0,"POLYGON ((-35.99556 11.09909, -35.98669 11.098..."
1,34.0,AL182021,al,18.0,4.0,2021092312,UTC,40.0,0.0,0.0,40.0,"POLYGON ((-37.39408 11.46545, -37.38223 11.465..."


In [9]:
ct = []
for tc in tc_list:
    df = NhcDownloaderBot(tc, 2021)
    df = df.gdf_file(tc, 2021)
    ct.append(NhcDownloaderBot.track_line_gdf)
    if tc == tc_list[-1]:
        print(f'--------------------done-----------------------')
    
#     NhcDownloaderBot.init_radii_gdf
    

--------------------done-----------------------


In [12]:
ct[0]

,STORMNAME,STORMTYPE,ADVDATE,ADVISNUM,STORMNUM,FCSTPRD,BASIN,geometry
0,Peter,TS,1100 AM AST Mon Sep 20 2021,8,16.0,120.0,AL,"LINESTRING (-60.90000 19.50000, -62.70000 20.1..."
